## Exploratory Data Analysis 

Importing Libraries 

In [16]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from src.utils import check_missing_data
from scripts.db_utils import connect, sql_to_dataframe

Load data

In [17]:
#opening the connection
conn = connect()

# load telegram post data for banks
query = """ SELECT * FROM public.telegram_post_performance  """
df_posts = sql_to_dataframe(conn, query)

# load BOA App review data 
query = """ SELECT * FROM public.google_play_reviews  """
df_reviews = sql_to_dataframe(conn, query)

# load BOA telegram channnel subscribers data
query = """ SELECT * FROM public.telegram_subscription_growth  """
df_subscribers = sql_to_dataframe(conn, query)

#closing the connection
conn.close()

Connecting..
All good, Connection successful!


In [18]:
df_posts.head()

,id,post_link,date,views,post_time,bank,time_of_day
0,430,https://t.me/tikvahethiopia/70382,2022-05-19 13:47:14,310300,13:47:14,#CBE,afternoon
1,431,https://t.me/tikvahethiopia/70383,2022-05-19 13:47:14,310300,13:47:14,#CBE,afternoon
2,432,https://t.me/tikvahethiopia/73378,2022-08-30 11:13:05,267400,11:13:05,#CBE,morning
3,433,https://t.me/tikvahethiopia/73379,2022-08-30 11:13:05,267400,11:13:05,#CBE,morning
4,434,https://t.me/tikvahethiopia/74556,2022-11-03 16:07:35,347100,16:07:35,#CBE,afternoon


In [19]:
df_reviews.head()

,id,review_id,username,user_image,likes,review_created_version,created_at,reply_content,replied_at,app_version,score,comments,keywords,lda_category,sentiment,insight
0,1,738b6d3a-2d7f-4a82-bfe2-dcf32591f944,NAHOM NIGUSSIE,https://play-lh.googleusercontent.com/a-/ALV-U...,1,None,2024-05-21 18:38:28,None,None,None,2,Slow and buggy crashed all the time need a lot...,None,None,None,None
1,2,e20abe49-8fe7-42fe-af3a-91399875b21a,Abbatu Ermias,https://play-lh.googleusercontent.com/a/ACg8oc...,0,24.04.23,2024-05-21 05:06:05,None,None,24.04.23,5,Awe ሃረፍ nawe,None,None,None,None
2,3,55c833c8-a942-47bb-aaaa-e022f39e28af,Habtsh Darge,https://play-lh.googleusercontent.com/a/ACg8oc...,0,None,2024-05-20 08:07:49,None,None,None,1,በጣም ቀፋፊ አፕ ነው ። ሰርቶ አያውቅም ። በጣም ብዙ ችግር አለበት ከአ...,None,None,None,None
3,4,d62d6d95-2900-4038-aa26-c9b42835c971,Usman,https://play-lh.googleusercontent.com/a/ACg8oc...,0,None,2024-05-19 16:27:13,None,None,None,1,This application is very bad . Please modify t...,None,None,None,None
4,5,20c4de51-f5cd-41f3-b21a-efdf8cb4463e,Eskender Million,https://play-lh.googleusercontent.com/a-/ALV-U...,1,24.04.23,2024-05-17 20:37:40,None,None,24.04.23,1,"It has a very complex, inconvenient to use, an...",None,None,None,None


In [20]:
df_subscribers

,id,channel_id,date,subscriber_count
0,1,None,2024-05-24,-1
1,2,None,2024-05-23,24
2,3,None,2024-05-22,-104
3,4,None,2024-05-21,-10
4,5,None,2024-05-20,104
...,...,...,...,...
140,141,None,2024-05-01,-150
141,142,None,2024-04-01,-24
142,143,None,2024-03-01,13
143,144,None,2024-02-01,584


Data Quality Assessment

In [23]:
# checking for missing values
missing_data_df_post = check_missing_data(df_posts)
print(missing_data_df_post)

   Column Name  Missing Values  Percentage Missing
2         date               1            0.549451
4    post_time               1            0.549451
6  time_of_day               1            0.549451


In [24]:
# checking for missing values
missing_data_df_reveiws = check_missing_data(df_reviews)
print(missing_data_df_reveiws)

               Column Name  Missing Values  Percentage Missing
5   review_created_version              87                29.0
7            reply_content             300               100.0
8               replied_at             300               100.0
9              app_version              87                29.0
12                keywords             300               100.0
13            lda_category             300               100.0
14               sentiment             300               100.0
15                 insight             300               100.0


In [25]:
# checking for missing values
missing_data_df_subscribers = check_missing_data(df_subscribers)
print(missing_data_df_subscribers)

  Column Name  Missing Values  Percentage Missing
1  channel_id             145               100.0


Data cleaning

In [26]:
df_subscribers.drop('channel_id', axis=1, inplace=True)

In [27]:
# checking for missing values
missing_data_df_subscribers = check_missing_data(df_subscribers)
print(missing_data_df_subscribers)

Success: No missing values.


In [28]:
df_posts.dropna(subset=['date', 'post_time', 'time_of_day'], inplace=True)

In [29]:
# checking for missing values
missing_data_df_post = check_missing_data(df_posts)
print(missing_data_df_post)

Success: No missing values.


In [32]:
df_reviews['review_created_version'] = df_reviews['review_created_version'].ffill()
df_reviews['app_version'] = df_reviews['app_version'].ffill()

In [33]:
# checking for missing values
missing_data_df_reveiws = check_missing_data(df_reviews)
print(missing_data_df_reveiws)

      Column Name  Missing Values  Percentage Missing
7   reply_content             300               100.0
8      replied_at             300               100.0
12       keywords             300               100.0
13   lda_category             300               100.0
14      sentiment             300               100.0
15        insight             300               100.0
